In [3]:
# Read Fashion MNIST dataset

import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

# Your code goes here . . .

data/fashion/train-labels-idx1-ubyte.gz
data/fashion/t10k-labels-idx1-ubyte.gz


In [4]:
print(y_train)
# print(X_test, y_test)

print(X_train.shape)
print(y_train.shape)

import matplotlib.pyplot as plt
#plot the first image in the dataset
print(X_train[0].shape)

import numpy as np
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 
encoder = OneHotEncoder(categories='auto')
y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.fit_transform(y_test.reshape(-1, 1)).toarray()
print(y_train.shape)
print(y_test.shape)

X_train = (X_train / 255)
X_test = (X_test / 255)


def sigmoid(z):
    return 1.0/(1 + np.exp(-z))

def sigmoid_derv(z):
    return z * (1 - z)

def softmax(s):
    exps = np.exp(s - np.max(s, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss
 

[9 0 0 ... 3 0 5]
(60000, 784)
(60000,)
(784,)
(60000, 10)
(10000, 10)


In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical
from keras.optimizers import Adam, Adadelta

X_train_conv = np.copy(X_train).reshape(60000,28,28,1)
X_test_conv = np.copy(X_test).reshape(10000,28,28,1)

filters = 8
filter_size = 3
pool_size = 2

convolutional_model = Sequential()
convolutional_model.add(Conv2D(filters, filter_size, input_shape=(28, 28, 1)))
convolutional_model.add(MaxPooling2D(pool_size = pool_size))
convolutional_model.add(Flatten())
convolutional_model.add(Dense(128, activation='sigmoid'))
convolutional_model.add(Dense(10, activation='softmax'))

convolutional_model.summary()

# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
convolutional_model.compile(optimizer=Adadelta(), loss='categorical_crossentropy',metrics=['accuracy'])

# Training
convolutional_model.fit(X_train_conv, y_train, epochs=10, validation_data=(X_train_conv, y_test))

scores = convolutional_model.evaluate(X_test_conv, y_test,)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print("\n%s: %.2f%%" % (convolutional_model.metrics_names[1], scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1352)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               173184    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 174,554
Trainable params: 174,554
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 77us/step - loss: 0.4492 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, Adadelta

model = Sequential()
model.add(Dense(128, input_dim= X_train.shape[1], activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(optimizer=Adadelta(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64)

scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:

class NeuralNetwork:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.neurons = 128
        self.learning_rate = 0.5
        self.theta1 = np.random.randn(self.x.shape[1], self.neurons)
        self.bias1 = np.zeros((1, self.neurons))
        self.theta2 = np.random.randn(self.neurons, self.y.shape[1])
        self.bias2 = np.zeros((1, self.y.shape[1]))
       
       
        
    # y = softmax(W2 sigmoid(w1x + b1) + b2)
    def forward_feeding(self):
        self.layer1 = sigmoid(np.dot(self.x, self.theta1) + self.bias1)
        self.output = softmax(np.dot(self.layer1, self.theta2) + self.bias2)
             
        
    def back_propagation(self, epoch_count):
        loss = error(self.output, self.y)
        if(epoch_count % 50 == 0):
             print('Error, epoch_count:', loss, epoch_count)
        
        a2_delta = cross_entropy(self.output, self.y) # theta2
              
        z1_delta = np.dot(a2_delta, self.theta2.T)
        a1_delta = z1_delta * sigmoid_derv(self.layer1) # theta1
        
        self.theta2 -= self.learning_rate * np.dot(self.layer1.T, a2_delta)
        self.bias2 -= self.learning_rate * np.sum(a2_delta, axis=0, keepdims=True)
        
        self.theta1 -= self.learning_rate * np.dot(self.x.T, a1_delta)
        self.bias1 -= self.learning_rate * np.sum(a1_delta, axis=0)
    
    def predict(self, data):
        self.x = data
        self.forward_feeding()
        return self.output.argmax()
        

model = NeuralNetwork(X_train, y_train)
epochs = 300
for x in range(epochs):
    model.forward_feeding()
    model.back_propagation(x)

def get_acc(x, y):
    acc = 0
    for xx,yy in zip(x, y):
        s = model.predict(xx)
        if s == np.argmax(yy):
            acc +=1
    return acc/len(x)*100

print("Training accuracy : ", get_acc(X_train, np.array(y_train)))
print("Test accuracy : ", get_acc(X_test, np.array(y_test)))